In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from make_more_efficient import *


def findRoundAbouts(datapath:str):
    allRoundabouts = []
    for i in range(0,15):
        folder_path = datapath + str(i)

        data_frames = []
        for file_name in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)

        data = pd.concat(data_frames, ignore_index=True)
        data = data[(data['speed'] >= 25) & (data['speed'] <= 45)]
        data['heading_rad'] = np.radians(data['heading'])
        data['vx'] = np.cos(data['heading_rad'])  
        data['vy'] = np.sin(data['heading_rad'])
        grid_lon = np.linspace(data['longitude'].min(), data['longitude'].max(), 100)
        grid_lat = np.linspace(data['latitude'].min(), data['latitude'].max(), 100)
        grid_lon, grid_lat = np.meshgrid(grid_lon, grid_lat)
        grid_vx = griddata((data['longitude'], data['latitude']), data['vx'], (grid_lon, grid_lat), method='linear')
        grid_vy = griddata((data['longitude'], data['latitude']), data['vy'], (grid_lon, grid_lat), method='linear')

        curl = np.gradient(grid_vy, axis=0) - np.gradient(grid_vx, axis=1)
        max_curl = np.nanmax(curl) 
        print("Maximum Curl Value:", max_curl)
        threshold =  0.9 * max_curl 
        near_max_curl_indices = np.argwhere(curl >= threshold)

        near_max_curl_points = []
        for idx in near_max_curl_indices:
            lon_idx, lat_idx = idx[1], idx[0]  
            lon = grid_lon[lon_idx][lat_idx]
            lat = grid_lat[lon_idx][lat_idx]
            near_max_curl_points.append((lat,lon))

        near_max_curl_df = pd.DataFrame(near_max_curl_points, columns=['Latitude',''])
        near_max_curl_df = pd.DataFrame(near_max_curl_points, columns=['Latitude','Longitude'])

        average_lat = near_max_curl_df['Latitude'].mean()
        average_lon = near_max_curl_df['Longitude'].mean()
        allRoundabouts.append((average_lat, average_lon))



Maximum Curl Value: 1.6289802770695891
Maximum Curl Value: 1.5100218471219475
Maximum Curl Value: 1.7661341797593961
Maximum Curl Value: 1.3879292974005162
Maximum Curl Value: 1.5677115638989825
Maximum Curl Value: 1.5516493998398841
Maximum Curl Value: 1.4051172326849573
Maximum Curl Value: 1.6516501113922488
Maximum Curl Value: 1.4747768598323454
Maximum Curl Value: 1.558362589089844
Maximum Curl Value: 1.4240142295106528
Maximum Curl Value: 1.5184691154476262
Maximum Curl Value: 1.603234160769151
Maximum Curl Value: 1.474486606209934
Maximum Curl Value: 1.5033507566343345
[(53.549155443992525, 9.960136950415801), (53.62829169407273, 10.147640025949919), (53.556297801229306, 10.04958506172926), (53.60720896255637, 9.856829465314913), (53.58905024611408, 10.045448158794379), (53.49730069832639, 10.01285609774633), (53.44021403959596, 10.306387097357195), (53.45703795837117, 9.991070414373942), (53.593449594706975, 10.11329620292839), (53.61809963447439, 9.907365987937936), (53.5018952